<a href="https://colab.research.google.com/github/tjhung23/MBC/blob/main/02_%C4%90%E1%BB%91i_So%C3%A1t_COD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install xlsxwriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
# lấy báo cáo đối soát cổng thanh toán
import glob
import os
import pandas as pd
import numpy as np
import csv
from datetime import datetime
import getpass
import pytz
import time
# --------------------------------------------------------------------------------------------------------------------------
# Lấy thời gian bắt đầu
start_time = time.time()

vn_tz = pytz.timezone('Asia/Ho_Chi_Minh')
now = datetime.now(vn_tz)

# Tạo timestamp
timestamp = now.strftime("%Y-%m-%d_%H-%M")
user = getpass.getuser()

#---------------------------------------------------------------------------------------------------------------------------
xlpath = '/content/drive/My Drive/KeToan_HachToanGiaoDich/CTT/2025/T06/'
result = glob.glob(xlpath + "*.*")
data = pd.DataFrame(columns=['Date', 'Ref', 'Amount', 'Bank'])
cuoc_data = 4500
cuoc_thoai = 500
cuoc_sms = 300
BIDV_list = []
ONEPAY_list = []
VNPAY_list = []
TCB_list = []
VNPOST_list = []
VTPOST_list = []
dwh_list = []
dwh_bh_list = []
#---------------------------------------------------------------------------------------------------------------------------
def generate_note(sim_type):
    if sim_type in ['e sim', 'esim']:
        return '338711 - Esim'
    elif sim_type == 'Sim vật lý':
        return '338711 - Sim vật lý'
    else:
        return 'Khác'
#---------------------------------------------------------------------------------------------------------------------------
for filename in result:
  # try:
    if filename.endswith((".xlsx", ".xls")):
      df = pd.read_excel(filename, dtype=str)
#---------------------------------------------------------------------------------------------------------------------------
      #DWH_doi soat CTT

      if df.columns[0] == 'Mã giao dịch':
        WH = df.copy()
        dwh = df.copy()
        dwh = dwh[dwh['Trạng thái thanh toán'] == 'Thành công']
        dwh = dwh[dwh['Cổng thanh toán']!= "MAIN_ACCOUNT"]

        cols = ['Giá trị đơn', 'Giảm giá', 'Phí thanh toán', 'Tổng tiền', 'Phí cổng thanh toán', 'Giá trị còn lại', 'SL data', 'SL SMS', 'SL Voice', 'Giá Data', 'Giá SMS', 'Giá Voice', 'Giá gói'] ## cột cần cast sang int
        dwh[cols] = dwh[cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
        # Tính giá mới cho đơn hàng, Logic của Kế toán
        dwh['Giá SMS2'] = dwh['SL SMS'] * cuoc_sms * ((dwh['Loại đơn hàng'] == 'Mua gói cước') & (dwh['Gói cước'] == 'TỰ DO TRẢI NGHIỆM')).astype(int)

        dwh['Giá Thoại'] = dwh.apply(lambda row: row['SL Voice'] * cuoc_thoai if row['Loại đơn hàng'] == 'Mua gói cước' and row['Gói cước'] == 'TỰ DO TRẢI NGHIỆM' else row['Giá trị đơn'] if row['Loại đơn hàng'] == 'Mua gói cước' and str(row['Gói cước']).startswith('V') else 0, axis=1)

        dwh['Giá Data2'] = dwh.apply(lambda row: row['Tổng tiền'] - row['Giá SMS2'] - row['Giá Thoại'] if row['Loại đơn hàng'] == 'Mua gói cước' and row['Gói cước'] == 'TỰ DO TRẢI NGHIỆM' else row['Tổng tiền'] if row['Loại đơn hàng'] == 'Mua gói cước' and str(row['Gói cước']).startswith('D') else row['Tổng tiền'] if row['Loại đơn hàng'] == 'VOUCHER_DATA' else 0, axis=1)

        dwh['Giá combo'] = dwh.apply(lambda row: row['Giá trị đơn'] - row['Giá Thoại'] - row['Giá Data2'] - row['Giá SMS2'] if pd.notnull(row['Giá trị đơn']) and pd.notnull(row['Giá Thoại']) and pd.notnull(row['Giá Data2']) and pd.notnull(row['Giá SMS2']) else 0, axis=1)
        dwh['Mã giao dịch'] = dwh['Mã giao dịch'].astype(str).str.strip()
        dwh_list.append(dwh) # Bỏ vào Rổ ONEPAY
        # # Create SDT_DAT
        # SDT_DAT = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['SĐT đặt']

        # # Map SDT_DAT to 'SDT đặt' => lấy SDT từ bên báo cáo D2C qua, vì báo cáo CTT là SDT khác witel
        # dwh['SDT đặt'] = dwh['Mã đơn hàng'].map(SDT_DAT)

        # # Create KYC
        # KYC = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Đã KYC']

        # # Map 'KYC' => để biết thuê bao mua có KYC chưa
        # dwh['KYC'] = dwh['Mã đơn hàng'].map(KYC)


        # # Create 'new' column based on non-NaN values in 'SDT đặt' after mapping
        # dwh['new'] = dwh['SDT đặt'].notna().map({True: 'new', False: ''})

        # # Apply fillna operations and convert to string
        # dwh['SDT đặt'] = dwh['SDT đặt'].fillna(dwh['SĐT khách hàng']).fillna(0).astype(str)

        # #Discount => lấy value discount nếu có
        # giam_gia = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Giảm giá']
        # dwh['Giảm giá'] = dwh['Mã đơn hàng'].map(giam_gia).fillna(0).astype(int)

#---------------------------------------------------------------------------------------------------------------------------
      #DWH_BH => báo cáo bán hàng D2C

      if df.columns[0] == 'Mã vận đơn MBC':
        dwh_bh = df.copy()
        # các cột cần cast sang int để tính
        cols = ['Giá trị đơn hàng', 'Giảm giá','Phí ship', 'Tổng tiền(đã chia)', 'Tổng tiền(chưa chia)', 'Giá bộ KIT sim trắng cước hòa mạng','Giá call',
                'Giá gói','Đã KYC','Tổng data','Giá data','Giảm giá data','Tổng tiền call','Tổng call','Tổng tiền data','Giảm giá call','Tổng SMS','Giá SMS','Giảm giá SMS','Tổng tiền SMS']
        dwh_bh[cols] = dwh_bh[cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
        # Tính giá mới cho đơn hàng
        dwh_bh['Giá data2'] = dwh_bh['Tổng data'] * cuoc_data * (dwh_bh['Tên gói'] == "TỰ DO TRẢI NGHIỆM").astype(int)
        dwh_bh['Giá thoại2'] = (dwh_bh['Tổng call']- dwh_bh['Giảm giá call']) * cuoc_thoai * (dwh_bh['Tên gói'] == "TỰ DO TRẢI NGHIỆM").astype(int)
        dwh_bh['Giá SMS2'] = (dwh_bh['Tổng SMS'] - dwh_bh['Giảm giá SMS']) * cuoc_sms * (dwh_bh['Tên gói'] == "TỰ DO TRẢI NGHIỆM").astype(int)
        dwh_bh['Chênh lệch'] = (dwh_bh['Giá gói'] - (dwh_bh['Giá data2'] + dwh_bh['Giá thoại2'] + dwh_bh['Giá SMS2'])) * (dwh_bh['Tên gói'] == "TỰ DO TRẢI NGHIỆM").astype(int)
        dwh_bh['Combo - 1 tháng'] = dwh_bh['Giá gói'] * (dwh_bh['Tên gói']!= "TỰ DO TRẢI NGHIỆM").astype(int)
        dwh_bh['Giá hòa mạng'] = np.where(dwh_bh['Giá bộ KIT sim trắng cước hòa mạng'] == 0, 0,np.where(dwh_bh['Giá bộ KIT sim trắng cước hòa mạng'] == 20000, 20000, 25000))
        dwh_bh['Giá sim'] = dwh_bh['Giá bộ KIT sim trắng cước hòa mạng'] - dwh_bh['Giá hòa mạng']
        dwh_bh_list.append(dwh_bh) # bỏ vào rổ


In [ ]:
# Tạo một Series ánh xạ từ dwh_bh
map_muasim = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Tên gói']

# Chỉ cập nhật nếu là "Mua sim" và cột Gói cước đang trống
mask_muasim = (dwh['Loại đơn hàng'] == 'Mua sim') & (dwh['Gói cước'].isna())

# Gán giá trị mới từ mapping
dwh.loc[mask_muasim, 'Gói cước'] = dwh.loc[mask_muasim, 'Mã đơn hàng'].map(map_muasim)


In [ ]:
        # Create SDT_DAT
        SDT_DAT = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['SĐT đặt']

        # Map SDT_DAT to 'SDT đặt' => lấy SDT từ bên báo cáo D2C qua, vì báo cáo CTT là SDT khác witel
        dwh['SDT đặt'] = dwh['Mã đơn hàng'].map(SDT_DAT)

        # Create KYC
        KYC = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Đã KYC']

        # Map 'KYC' => để biết thuê bao mua có KYC chưa
        dwh['KYC'] = dwh['Mã đơn hàng'].map(KYC)


        # Create 'new' column based on non-NaN values in 'SDT đặt' after mapping
        dwh['new'] = dwh['SDT đặt'].notna().map({True: 'new', False: ''})

        # Apply fillna operations and convert to string
        dwh['SDT đặt'] = dwh['SDT đặt'].fillna(dwh['SĐT khách hàng']).fillna(0).astype(str)

        #Discount => lấy value discount nếu có
        giam_gia = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Giảm giá']
        dwh['Giảm giá'] = dwh['Mã đơn hàng'].map(giam_gia).fillna(0).astype(int)

if dwh_list:
    dwh = pd.concat(dwh_list, ignore_index=True)
    dwh = dwh.drop_duplicates(keep='first')  # Loại trùng toàn bộ dòng

if dwh_bh_list:
    dwh_bh = pd.concat(dwh_bh_list, ignore_index=True)
    dwh_bh = dwh_bh.drop_duplicates(keep='first')  # Loại trùng toàn bộ dòng

In [ ]:
# đặt map SDT, KYC, Giảm giá ngòi loop để code nhanh hơn
SDT= dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['SDT đặt']
KYC_check = dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['KYC']
giam_gia = dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giảm giá']
# Create SDT_DAT
SDT_DAT = dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['SĐT đặt']

# Map SDT_DAT to 'SDT đặt' => lấy SDT từ bên báo cáo D2C qua, vì báo cáo CTT là SDT khác witel
dwh['SDT đặt'] = dwh['Mã đơn hàng'].map(SDT_DAT)

In [ ]:
### load cổng thanh toán và map đơn hàng
result_bank = glob.glob(xlpath + "*.*")
#---------------------------------------------------------------------------------------------------------------------------
for filename in result_bank:
  try:
    if filename.endswith((".xlsx", ".xls")):
      df = pd.read_excel(filename, dtype=str)

# #---------------------------------------------------------------------------------------------------------------------------
      if df.columns[0] == 'TỔNG CÔNG TY CỔ PHẦN BƯU CHÍNH VIETTEL':
        tmp_VTP = df.copy()
        # Tìm index của dòng chứa "III:KẾT LUẬN ĐỐI SOÁT"
        end_index = tmp_VTP[tmp_VTP.iloc[:, 0].astype(str).str.contains('II:CHI TIẾT TIỀN CƯỚC CHUYỂN PHÁT VÀ PHÍ COD', na=False)].index.min()

        # Cắt từ dòng 11 đến dòng trước đó
        tmp_VTP = tmp_VTP.iloc[11:end_index, :-3]
        tmp_VTP.columns = tmp_VTP.iloc[0]
        tmp_VTP = tmp_VTP[1:].reset_index(drop=True)
        tmp_VTP = tmp_VTP.dropna(subset=['STT'])
        tmp_VTP.loc[:, 'Ngày phát thành công'] = tmp_VTP['Ngày phát thành công'].str.split(' ').str[0].str.replace('/', '-', regex=False)
        tmp_VTP.loc[:, 'Ngày nhập máy'] = tmp_VTP['Ngày nhập máy'].str.split(' ').str[0].str.replace('/', '-', regex=False)
        tmp_VTP['Số tiền COD'] = pd.to_numeric(tmp_VTP['Số tiền COD'].str.replace(',', '', regex=False), errors='coerce')
        # tmp_VTP['Số tiền COD'] = pd.to_numeric(tmp_VTP['Số tiền COD'], errors='coerce')
        tmp_VTP['SDT'] = tmp_VTP['Mã tham chiếu'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['SĐT đặt']).fillna('')
        tmp_VTP['KYC'] = tmp_VTP['Mã tham chiếu'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Đã KYC']).fillna('')
        tmp_VTP['Giảm giá'] = tmp_VTP['Mã tham chiếu'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Giảm giá']).fillna(0).astype(int)
        tmp_VTP['Gói cước'] = tmp_VTP['Mã tham chiếu'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Tên gói']).fillna('')
        tmp_VTP['Loại đơn hàng'] = tmp_VTP['Mã tham chiếu'].astype(str).str[:2].apply(lambda x: 'Mua sim' if x == 'OD' else 'Đổi sim')
        tmp_VTP['Loại sim'] = tmp_VTP['Mã tham chiếu'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Loại sim']).fillna('0')

        tmp_VTP_affiliate = tmp_VTP['Mã tham chiếu'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Affiliate partner'])
        tmp_VTP['Đơn vị'] = np.where(tmp_VTP_affiliate == 'ACCESSTRADE','SALES - Online - ACT','SALES - Online')
        tmp_VTP['Tổng tiền thu'] = tmp_VTP['Số tiền COD']

        so_luong_sim = dwh_bh['Mã vận đơn MBC'].value_counts()
        tmp_VTP['Số lượng sim'] = tmp_VTP['Mã tham chiếu'].map(so_luong_sim).fillna(0).astype(int)

        # 1. Chọn các dòng có Phí hòa mạng > 0
        mask_hoa_mang = dwh_bh['Giá hòa mạng'] > 0

        # 2. Đếm số lần mỗi Mã vận đơn MBC phát sinh Phí hòa mạng
        so_luong_hoa_mang = (dwh_bh.loc[mask_hoa_mang, 'Mã vận đơn MBC'].value_counts())

        # 3. Map vào tmp_VTP_MISA
        tmp_VTP['Số lượng hòa mạng'] = (tmp_VTP['Mã tham chiếu'].map(so_luong_hoa_mang).fillna(0).astype(int))

        tong_gia_sim = dwh_bh.groupby('Mã vận đơn MBC')['Giá sim'].sum()
        tmp_VTP['Tiền sim trắng'] = np.where(tmp_VTP['Loại đơn hàng'] == "Đổi sim",5000,tmp_VTP['Mã tham chiếu'].map(tong_gia_sim).fillna(0))

        tong_gia_hoa_mang = dwh_bh.groupby('Mã vận đơn MBC')['Giá hòa mạng'].sum()
        tmp_VTP['Phí hòa mạng'] = np.where(tmp_VTP['Loại đơn hàng'] == "Mua sim",tmp_VTP['Mã tham chiếu'].map(tong_gia_hoa_mang).fillna(0),0)

        tong_gia_thoai = dwh_bh.groupby('Mã vận đơn MBC')['Giá thoại2'].sum()
        tmp_VTP['Tiền gói - Thoại'] = np.where(tmp_VTP['Loại đơn hàng'] == "Mua gói cước", tmp_VTP['Mã tham chiếu'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá Thoại']).fillna(0).astype(int),
                                    np.where(tmp_VTP['Loại đơn hàng'] == "Mua sim",tmp_VTP['Mã tham chiếu'].map(tong_gia_thoai).fillna(0),0))

        tong_gia_sms = dwh_bh.groupby('Mã vận đơn MBC')['Giá SMS2'].sum()
        tmp_VTP['Tiền gói - SMS'] = np.where(tmp_VTP['Loại đơn hàng'] == "Mua gói cước", tmp_VTP['Mã tham chiếu'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá SMS2']).fillna(0).astype(int),
                                    np.where(tmp_VTP['Loại đơn hàng'] == "Mua sim",tmp_VTP['Mã tham chiếu'].map(tong_gia_sms).fillna(0),0))

        tong_gia_data = dwh_bh.groupby('Mã vận đơn MBC')['Giá data2'].sum()

        tmp_VTP['Tiền gói - Data'] = np.where(tmp_VTP['Loại đơn hàng'] == "Mua gói cước",tmp_VTP['Mã tham chiếu'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá Data2']).fillna(0).astype(int),
                                    np.where(tmp_VTP['Loại đơn hàng'] == "Mua sim",tmp_VTP['Mã tham chiếu'].map(tong_gia_data).fillna(0),
                                    np.where(tmp_VTP['Loại đơn hàng'] == "VOUCHER_DATA",tmp_VTP['Tổng tiền thu'].fillna(0),0)))

        tong_gia_combo = dwh_bh.groupby('Mã vận đơn MBC')['Combo - 1 tháng'].sum()
        tmp_VTP['Tiền gói - Combo 1 tháng'] = np.where(tmp_VTP['Loại đơn hàng'] == "Mua gói cước", tmp_VTP['Mã tham chiếu'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá combo']).fillna(0).astype(int),
                                    np.where(tmp_VTP['Loại đơn hàng'] == "Mua sim",tmp_VTP['Mã tham chiếu'].map(tong_gia_combo).fillna(0),0))

        tmp_VTP['Doanh thu khác'] = np.where(tmp_VTP['Loại đơn hàng'] == "Đổi sim",20000,np.where(tmp_VTP['Loại đơn hàng'] == "Khôi phục số",tmp_VTP['Tổng tiền thu'],0))

        tmp_VTP['Tiền ship'] = np.where(tmp_VTP['Loại đơn hàng'] == "Đổi sim",tmp_VTP['Tổng tiền thu'] - tmp_VTP['Tiền sim trắng'] - tmp_VTP['Doanh thu khác'],
                                    np.where(tmp_VTP['Loại đơn hàng'] == "Mua sim",tmp_VTP['Mã tham chiếu'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Phí ship']).fillna(0),0))

        tmp_VTP['Topup'] = np.where(tmp_VTP['Loại đơn hàng'] == "Topup",tmp_VTP['Tổng tiền thu'],0)

        tmp_VTP['Voucher'] = np.where(tmp_VTP['Loại đơn hàng'] == "VOUCHER",tmp_VTP['Tổng tiền thu'],0)

        cols_to_subtract = ['Tiền sim trắng', 'Phí hòa mạng', 'Tiền gói - Thoại', 'Tiền gói - SMS',    'Tiền gói - Data', 'Tiền gói - Combo 1 tháng', 'Doanh thu khác','Tiền ship', 'Topup', 'Voucher']
        tmp_VTP['Check'] = tmp_VTP['Tổng tiền thu'] - tmp_VTP[cols_to_subtract].sum(axis=1)
        VTPOST_list.append(tmp_VTP) # bỏ vào rổ VTPOST

# #---------------------------------------------------------------------------------------------------------------------------

      elif df.iloc[0, 0] == 'BẢNG KÊ CƯỚC THU HỘ MOBICAST - VNPOST LOGISTICS':
        tmp_VNP = df.copy()
        end_index = tmp_VNP[tmp_VNP.iloc[:, 0].astype(str).str.contains('Tổng', na=False)].index.min()
        tmp_VNP = tmp_VNP.iloc[3:end_index, :-1]
        tmp_VNP.columns = tmp_VNP.iloc[0]
        tmp_VNP = tmp_VNP[1:].reset_index(drop=True)
        tmp_VNP['Ngày thu tiền'] =tmp_VNP['Ngày thu tiền'].astype(str).str.split(" ").str[0].str.replace('/', '-', regex=False)
        tmp_VNP['Số tiền'] = pd.to_numeric(tmp_VNP['Số tiền'], errors='coerce')
        tmp_VNP['SDT'] = tmp_VNP['Số lô/bảng kê'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['SĐT đặt']).fillna('')
        tmp_VNP['KYC'] = tmp_VNP['Số lô/bảng kê'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Đã KYC']).fillna('')
        tmp_VNP['Giảm giá'] = tmp_VNP['Số lô/bảng kê'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Giảm giá']).fillna(0).astype(int)
        tmp_VNP['Gói cước'] = tmp_VNP['Số lô/bảng kê'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Tên gói']).fillna('')
        tmp_VNP['Loại đơn hàng'] = tmp_VNP['Số lô/bảng kê'].astype(str).str[:2].apply(lambda x: 'Mua sim' if x == 'OD' else 'Đổi sim')
        tmp_VNP['Số tiền'] = pd.to_numeric(tmp_VNP['Số tiền'], errors='coerce')
        tmp_VNP['Loại sim'] = tmp_VNP['Số lô/bảng kê'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Loại sim']).fillna('0')

        tmp_VNP_affiliate = tmp_VNP['Số lô/bảng kê'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Affiliate partner'])
        tmp_VNP['Đơn vị'] = np.where(tmp_VNP_affiliate == 'ACCESSTRADE','SALES - Online - ACT','SALES - Online')
        tmp_VNP['Tổng tiền thu'] = tmp_VNP['Số tiền']

        so_luong_sim = dwh_bh['Mã vận đơn MBC'].value_counts()
        tmp_VNP['Số lượng sim'] = tmp_VNP['Số lô/bảng kê'].map(so_luong_sim).fillna(0).astype(int)

        # 1. Chọn các dòng có Phí hòa mạng > 0
        mask_hoa_mang = dwh_bh['Giá hòa mạng'] > 0

        # 2. Đếm số lần mỗi Mã vận đơn MBC phát sinh Phí hòa mạng
        so_luong_hoa_mang = (dwh_bh.loc[mask_hoa_mang, 'Mã vận đơn MBC'].value_counts())

        # 3. Map vào tmp_VNP_MISA
        tmp_VNP['Số lượng hòa mạng'] = (tmp_VNP['Số lô/bảng kê'].map(so_luong_hoa_mang).fillna(0).astype(int))

        tong_gia_sim = dwh_bh.groupby('Mã vận đơn MBC')['Giá sim'].sum()
        tmp_VNP['Tiền sim trắng'] = np.where(tmp_VNP['Loại đơn hàng'] == "Đổi sim",5000,tmp_VNP['Số lô/bảng kê'].map(tong_gia_sim).fillna(0))

        tong_gia_hoa_mang = dwh_bh.groupby('Mã vận đơn MBC')['Giá hòa mạng'].sum()
        tmp_VNP['Phí hòa mạng'] = np.where(tmp_VNP['Loại đơn hàng'] == "Mua sim",tmp_VNP['Số lô/bảng kê'].map(tong_gia_hoa_mang).fillna(0),0)

        tong_gia_thoai = dwh_bh.groupby('Mã vận đơn MBC')['Giá thoại2'].sum()
        tmp_VNP['Tiền gói - Thoại'] = np.where(tmp_VNP['Loại đơn hàng'] == "Mua gói cước", tmp_VNP['Số lô/bảng kê'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá Thoại']).fillna(0).astype(int),
                                    np.where(tmp_VNP['Loại đơn hàng'] == "Mua sim",tmp_VNP['Số lô/bảng kê'].map(tong_gia_thoai).fillna(0),0))

        tong_gia_sms = dwh_bh.groupby('Mã vận đơn MBC')['Giá SMS2'].sum()
        tmp_VNP['Tiền gói - SMS'] = np.where(tmp_VNP['Loại đơn hàng'] == "Mua gói cước", tmp_VNP['Số lô/bảng kê'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá SMS2']).fillna(0).astype(int),
                                    np.where(tmp_VNP['Loại đơn hàng'] == "Mua sim",tmp_VNP['Số lô/bảng kê'].map(tong_gia_sms).fillna(0),0))

        tong_gia_data = dwh_bh.groupby('Mã vận đơn MBC')['Giá data2'].sum()

        tmp_VNP['Tiền gói - Data'] = np.where(tmp_VNP['Loại đơn hàng'] == "Mua gói cước",tmp_VNP['Số lô/bảng kê'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá Data2']).fillna(0).astype(int),
                                    np.where(tmp_VNP['Loại đơn hàng'] == "Mua sim",tmp_VNP['Số lô/bảng kê'].map(tong_gia_data).fillna(0),
                                    np.where(tmp_VNP['Loại đơn hàng'] == "VOUCHER_DATA",tmp_VNP['Tổng tiền thu'].fillna(0),0)))

        tong_gia_combo = dwh_bh.groupby('Mã vận đơn MBC')['Combo - 1 tháng'].sum()
        tmp_VNP['Tiền gói - Combo 1 tháng'] = np.where(tmp_VNP['Loại đơn hàng'] == "Mua gói cước", tmp_VNP['Số lô/bảng kê'].map(dwh.drop_duplicates('Mã đơn hàng').set_index('Mã đơn hàng')['Giá combo']).fillna(0).astype(int),
                                    np.where(tmp_VNP['Loại đơn hàng'] == "Mua sim",tmp_VNP['Số lô/bảng kê'].map(tong_gia_combo).fillna(0),0))

        tmp_VNP['Doanh thu khác'] = np.where(tmp_VNP['Loại đơn hàng'] == "Đổi sim",20000,np.where(tmp_VNP['Loại đơn hàng'] == "Khôi phục số",tmp_VNP['Tổng tiền thu'],0))

        tmp_VNP['Tiền ship'] = np.where(tmp_VNP['Loại đơn hàng'] == "Đổi sim",tmp_VNP['Tổng tiền thu'] - tmp_VNP['Tiền sim trắng'] - tmp_VNP['Doanh thu khác'],
                                    np.where(tmp_VNP['Loại đơn hàng'] == "Mua sim",tmp_VNP['Số lô/bảng kê'].map(dwh_bh.drop_duplicates('Mã vận đơn MBC').set_index('Mã vận đơn MBC')['Phí ship']).fillna(0),0))

        tmp_VNP['Topup'] = np.where(tmp_VNP['Loại đơn hàng'] == "Topup",tmp_VNP['Tổng tiền thu'],0)

        tmp_VNP['Voucher'] = np.where(tmp_VNP['Loại đơn hàng'] == "VOUCHER",tmp_VNP['Tổng tiền thu'],0)

        cols_to_subtract = ['Tiền sim trắng', 'Phí hòa mạng', 'Tiền gói - Thoại', 'Tiền gói - SMS',    'Tiền gói - Data', 'Tiền gói - Combo 1 tháng', 'Doanh thu khác','Tiền ship', 'Topup', 'Voucher']
        tmp_VNP['Check'] = tmp_VNP['Tổng tiền thu'] - tmp_VNP[cols_to_subtract].sum(axis=1)
        VNPOST_list.append(tmp_VNP) # bỏ vào rổ VTPOST

  except Exception as e: # Bắt lỗi
    print(f"❌ Lỗi xử lý file: {filename}") # File bị lỗi
    print(f"Chi tiết lỗi: {e}") # Tên Lỗi
# Concat data các cổng thanh toán


if VNPOST_list:
    VNPOST = pd.concat(VNPOST_list, ignore_index=True)

if VTPOST_list:
    VTPOST = pd.concat(VTPOST_list, ignore_index=True)



In [ ]:
print(VNPOST.shape)
print(VTPOST.shape)

(40, 28)
(117, 32)


In [ ]:
VNPOST_REC = VNPOST.copy()
VNPOST_REC = VNPOST_REC[~(VNPOST_REC['Check'] != 0)]
VNPOST_nan = VNPOST.copy()
VNPOST_nan = VNPOST_nan[(VNPOST_nan['Check']!= 0)]
VNPOST_nan = VNPOST_nan.rename(columns={'Số lô/bảng kê' : 'REF','Ngày thu tiền' : 'Date'})

In [ ]:
VTPOST_REC = VTPOST.copy()
VTPOST_REC = VTPOST_REC[~(VTPOST_REC['Check'] != 0)]
VTPOST_nan = VTPOST.copy()
VTPOST_nan = VTPOST_nan[(VTPOST_nan['Check']!= 0)]
VTPOST_nan = VTPOST_nan.rename(columns={'Mã tham chiếu' : 'REF','Ngày phát thành công' : 'Date'})

In [ ]:
#lấy columns của df cần check lại với IT
columns_nan = [ 'Date', 'REF','SDT','KYC', 'Tổng tiền thu', 'Loại đơn hàng', 'Gói cước', 'Loại sim', 'Đơn vị', 'Số lượng sim', 'Số lượng hòa mạng', 'Tiền sim trắng',
               'Phí hòa mạng', 'Tiền gói - Thoại', 'Tiền gói - SMS', 'Tiền gói - Data', 'Tiền gói - Combo 1 tháng', 'Doanh thu khác', 'Tiền ship', 'Topup', 'Voucher','Check','Giảm giá','Source']

In [ ]:
VNPOST_nan['Source'] = 'VNPOST'
VTPOST_nan['Source'] = 'VTPOST'
DCHECK = pd.concat([VNPOST_nan,VTPOST_nan], ignore_index=True)


In [ ]:
# lấy columns của df đã reconcile
columns_misa = [ 'Date','SDT','KYC', 'Tổng tiền thu', 'Loại đơn hàng', 'Gói cước', 'Loại sim', 'Đơn vị', 'Số lượng sim', 'Số lượng hòa mạng',
 'Tiền sim trắng', 'Phí hòa mạng', 'Tiền gói - Thoại', 'Tiền gói - SMS', 'Tiền gói - Data', 'Tiền gói - Combo 1 tháng', 'Doanh thu khác', 'Tiền ship', 'Topup', 'Voucher','Source']

In [ ]:
## Xử lý gộp df đã reconcile

In [ ]:
VNPOST_MISA = VNPOST_REC.copy()
VNPOST_MISA['KYC'] = VNPOST_MISA['KYC'].fillna(1)
VNPOST_MISA['Source'] = "VNPOST"
VNPOST_MISA = VNPOST_MISA.rename(columns = {'Ngày thu tiền' : 'Date'})
VNPOST_MISA = VNPOST_MISA[columns_misa]
VNPOST_MISA_group = VNPOST_MISA.groupby(['Date', 'Gói cước','Đơn vị']).sum(numeric_only=True).reset_index()
VNPOST_MISA_group['Số lượng'] = VNPOST_MISA.groupby(['Date', 'Gói cước','Đơn vị']).size().values
# VNPOST_MISA_group['Tổng tiền thu'].sum()

In [ ]:
VTPOST_MISA = VTPOST_REC.copy()
VTPOST_MISA['KYC'] = VTPOST_MISA['KYC'].fillna(1)
VTPOST_MISA['Source'] = "VTPOST"
VTPOST_MISA = VTPOST_MISA.rename(columns = {'Ngày phát thành công' : 'Date'})
VTPOST_MISA = VTPOST_MISA[columns_misa]
VTPOST_MISA_group = VTPOST_MISA.groupby(['Date', 'Gói cước','Đơn vị']).sum(numeric_only=True).reset_index()
VTPOST_MISA_group['Số lượng'] = VTPOST_MISA.groupby(['Date', 'Gói cước','Đơn vị']).size().values

# VTPOST_MISA_group['Tổng tiền thu'].sum()

In [ ]:
# Tính Số lượng sim và hòa mạng
df_sim = pd.concat([VNPOST_MISA,VTPOST_MISA], ignore_index=True)
df_sim = df_sim[df_sim['Loại sim'].isin(['Esim', 'Sim vật lý', 'E sim'])]
df_sim = df_sim.copy()
df_sim["Loại sim"] = df_sim["Loại sim"].str.replace("E sim", "Esim")

df_sim = df_sim[['Date','SDT','Loại sim', 'Đơn vị', 'Số lượng sim', 'Số lượng hòa mạng', 'Tiền sim trắng','Source']]

In [ ]:
df_sim_group = (
    df_sim
    .groupby(['Date', 'Loại sim','Source','Đơn vị'], as_index=False)
    .sum(numeric_only=True)  # Chỉ tính tổng các cột số
)

In [ ]:
Sim_melt = pd.melt(
    df_sim_group,
    id_vars=['Date', 'Loại sim', 'Đơn vị','Source'],  # giữ nguyên các cột này
    value_vars=["Số lượng sim","Số lượng hòa mạng", "Tiền sim trắng"],
    var_name='Type',   # tên cột mới cho loại
    value_name='Amount'         # tên cột mới cho giá trị
)
Sim_melt = Sim_melt[Sim_melt['Amount'] != 0]

Sim_melt['Non-VAT'] = np.where(
    Sim_melt['Type'].isin(["Số lượng hòa mạng","Số lượng sim"]),
    np.nan,
    Sim_melt['Amount'] / 1.1
)
Sim_melt['VAT'] = np.where(
    Sim_melt['Type'].isin(["Số lượng hòa mạng","Số lượng sim"]),
    np.nan,
    Sim_melt['Amount'] - Sim_melt['Non-VAT']
)

In [ ]:
Sim_melt["Tài khoản"] = np.where(
    Sim_melt["Type"] == "Tiền sim trắng",
    "338711",
    ""
)


In [ ]:
VNPOST_MISA_group['Source'] = 'VNPOST'
VTPOST_MISA_group['Source'] = 'VTPOST'
MISA = pd.concat([VNPOST_MISA_group,VTPOST_MISA_group], ignore_index=True)
MISA['Gói cước'] = MISA['Gói cước'].replace('TỰ DO TRẢI NGHIỆM', 'TU DO TRAI NGHIEM')
print(MISA.shape)



(62, 19)


In [ ]:
# check số trước và sau khi gộp
MISA['Tổng tiền thu'].sum()-(MISA[['Tiền sim trắng',    'Phí hòa mạng',    'Tiền gói - Thoại',    'Tiền gói - SMS',
    'Tiền gói - Data',    'Tiền gói - Combo 1 tháng',    'Doanh thu khác',    'Tiền ship',    'Topup','Voucher']].sum().sum())

np.int64(0)

In [ ]:
# các cột cần gộp giá trị
value_vars = [
    'Tổng tiền thu',
    'Số lượng sim',
    'Số lượng hòa mạng',
    'Tiền sim trắng',
    'Phí hòa mạng',
    'Tiền gói - Thoại',
    'Tiền gói - SMS',
    'Tiền gói - Data',
    'Tiền gói - Combo 1 tháng',
    'Doanh thu khác',
    'Tiền ship',
    'Topup',
    'Voucher',
]


MISA_melt = pd.melt(
    MISA,
    id_vars=['Date', 'Gói cước', 'Source','Đơn vị','Số lượng'],  # giữ nguyên các cột này
    value_vars=value_vars,
    var_name='Type',   # tên cột mới cho loại
    value_name='Amount'         # tên cột mới cho giá trị
)


In [ ]:
loai_can_dem = ['Tiền gói - Thoại', 'Tiền gói - SMS', 'Tiền gói - Data','Tiền gói - Combo 1 tháng']
# Gán lại Số_lượng_giao_dịch: chỉ giữ giá trị nếu Type thuộc nhóm

# Ép về str để xử lý đồng bộ
MISA_melt['Gói cước'] = MISA_melt['Gói cước'].astype(str)

# Clear các giá trị không hợp lệ về NaN
MISA_melt.loc[
    (MISA_melt['Gói cước'].str.lower() == 'nan') |
    (MISA_melt['Gói cước'] == '')|
    (MISA_melt['Gói cước'] == ' '),
    'Gói cước'
] = np.nan

MISA_melt['Số lượng'] = np.where(
    (MISA_melt['Type'].isin(loai_can_dem)) &
    (MISA_melt['Gói cước'].notna()),
    MISA_melt['Số lượng'],
    np.nan
)
# loai_goi = ['Tiền gói - Thoại', 'Tiền gói - SMS', 'Tiền gói - Data', 'Tiền gói - Combo 1 tháng']

MISA_melt['Gói cước'] = MISA_melt.apply(
    lambda row: row['Gói cước'] if row['Type'] in loai_can_dem else "",
    axis=1
)


In [ ]:
MISA_melt = MISA_melt[(MISA_melt['Amount'].notna()) &(MISA_melt['Amount'] != 0)]
MISA_melt = MISA_melt.copy()
MISA_melt['Số lượng'] = MISA_melt['Số lượng'].astype(float)


# MISA_melt["Type"].unique()

In [ ]:
MISA_melt[MISA_melt['Type'].isin(['Số lượng hòa mạng','Tiền sim trắng', 'Phí hòa mạng', 'Tiền gói - Thoại','Tiền gói - SMS', 'Tiền gói - Data', 'Tiền gói - Combo 1 tháng','Doanh thu khác', 'Tiền ship', 'Topup', 'Voucher','Số_lượng_giao_dịch'])]['Amount'].sum()

np.int64(16850655)

In [ ]:
# Convert sang tài khoản
mapping_dict = {
    'Phí hòa mạng': '338712',
    'Tiền gói - Thoại': '338721',
    'Tiền gói - SMS': '338722',
    'Tiền gói - Data': '338723',
    'Tiền ship': '33875',
    'Topup': '3387311',
    'Doanh thu khác': '51185',
    'Voucher': '51113',
    'Tiền gói - Combo 1 tháng': '3387241',
}


In [ ]:
# Loại những loại k đưa vào MISA
MISA_melt = MISA_melt[~MISA_melt['Type'].isin(["Tổng tiền thu","Số lượng hòa mạng","Số lượng sim","Tiền sim trắng"])].copy()
# Map tài khoản
MISA_melt['Tài khoản'] = MISA_melt['Type'].map(mapping_dict)
MISA_melt['Số lượng'] = MISA_melt['Số lượng'].fillna(1)


In [ ]:
MISA_melt[MISA_melt['Type'] == 'Tiền gói - Data'].sort_values(by='Source')

,Date,Gói cước,Source,Đơn vị,Số lượng,Type,Amount,Tài khoản
436,17-06-2025,TU DO TRAI NGHIEM,VNPOST,SALES - Online,1.0,Tiền gói - Data,45000,338723
454,21-06-2025,TU DO TRAI NGHIEM,VNPOST,SALES - Online,1.0,Tiền gói - Data,45000,338723
492,23-06-2025,TU DO TRAI NGHIEM,VTPOST,SALES - Online,1.0,Tiền gói - Data,45000,338723


In [ ]:
# Gộp lại theo ngày, gói, tài khoản, TYpe
MISA_melt2 =  (
    MISA_melt
    .groupby(['Date', 'Gói cước','Source','Đơn vị','Type','Tài khoản'],dropna = False, as_index=False)
    .sum(numeric_only=True)  # Chỉ tính tổng các cột số
)

In [ ]:
MISA_melt2[MISA_melt2['Type'] == 'Voucher'].sort_values(by='Source')

,Date,Gói cước,Source,Đơn vị,Type,Tài khoản,Số lượng,Amount


In [ ]:
# loại số lượng count không đúng
MISA_melt2['Số lượng'] = np.where(
    (MISA_melt2['Gói cước'] == "TỰ DO TRẢI NGHIỆM") & (MISA_melt2['Type'] != "Tiền gói - Data"),
    np.nan,
    MISA_melt2['Số lượng']
)

MISA_melt2['Số lượng'] = MISA_melt2['Số lượng'].replace(0, np.nan)


In [ ]:
#Tính VAT và Non-VAT
MISA_melt2['Non-VAT'] = np.where(MISA_melt2['Type'] == "Voucher", MISA_melt2['Amount'],
                                  MISA_melt2['Amount']/1.1)
MISA_melt2['VAT'] = MISA_melt2['Amount'] - MISA_melt2['Non-VAT']
MISA_melt2['VAT'] = np.where(MISA_melt2['Type'] == 'Voucher', np.nan, MISA_melt2['VAT'])


In [ ]:
MISA_melt2[MISA_melt2['Type'] == 'Voucher']

,Date,Gói cước,Source,Đơn vị,Type,Tài khoản,Số lượng,Amount,Non-VAT,VAT


In [ ]:
MISA_Excel = pd.concat([MISA_melt2,Sim_melt], ignore_index=True)

In [ ]:
#Generate mã chứng từ
MISA_Excel['Số chứng từ'] = (
    MISA_Excel['Source'] +
    MISA_Excel['Date'].str.split('-').str[2].str[-2:] +
    MISA_Excel['Date'].str.split('-').str[1] +
    MISA_Excel['Date'].str.split('-').str[0]
)

# Tạo STT bằng rank (kiểu int)
MISA_Excel['STT'] = np.where(
    MISA_Excel['Type'] == 'Voucher',
    2,
    1
)


# Ghép lại
MISA_Excel['Số chứng từ'] = (
    MISA_Excel['Số chứng từ'] + "-" +
    MISA_Excel['STT'].astype(str)
)



In [ ]:
# Generate diễn giải hạch toán
MISA_Excel['Diễn giải'] = 'Bán hàng Khách hàng không lấy hóa đơn - TT qua ' + MISA_Excel['Source'] + ' ngày ' + MISA_Excel['Date'] + '-' + MISA_Excel['Type']
MISA_Excel['Mã KH'] = np.where(MISA_Excel['Source'] == 'VTPOST', '000404',np.where(MISA_Excel['Source'] == 'VNPOST', '000455', np.nan))

# MISA_Excel['Mã KH'] = MISA_Excel['Source'] + '-KL'
MH_mapping = {
    'Topup': 'DVVT',
    'Phí hòa mạng': 'HM',
    'Doanh thu khác': 'DV',
    'Tiền ship': 'VC',
    'Tiền sim trắng': 'ST',
    'Voucher': 'VCPL',
}

# Danh sách các Type sẽ lấy theo Gói cước
goi_cuoc_types = [
    'Tiền gói - Thoại',
    'Tiền gói - SMS',
    'Tiền gói - Data',
    'Tiền gói - Combo 1 tháng'
]

def get_ma_hang(row):
    if row['Type'] in MH_mapping:
        return MH_mapping[row['Type']]
    elif row['Type'] in goi_cuoc_types:
        return row['Gói cước']
    else:
        return ''

MISA_Excel['Mã hàng'] = MISA_Excel.apply(get_ma_hang, axis=1)


In [ ]:
# - Lấy template input Misa
template = '/content/drive/My Drive/KeToan_HachToanGiaoDich/template/TemplateMISA.xls'
df_template = pd.read_excel(template)

In [ ]:
#Thao tác cho file input misa
df_template['Hiển thị trên sổ'] = '0'
df_template['Kiêm phiếu xuất kho'] = '0'
df_template['Ngày chứng từ (*)'] = MISA_Excel['Date']
df_template['TK Tiền/Chi phí/Nợ (*)'] = '1311'
df_template['TK Doanh thu/Có (*)'] = MISA_Excel['Tài khoản']
df_template['% thuế GTGT'] = '10'
df_template['Thành tiền'] = MISA_Excel['Non-VAT']
df_template['Tiền thuế GTGT'] = MISA_Excel['VAT']
df_template['TK thuế GTGT'] = '33311'
df_template['Đơn vị'] = MISA_Excel['Đơn vị']
df_template['Mã thống kê'] = "N"
df_template['Số chứng từ (*)'] = MISA_Excel['Số chứng từ']
df_template['Mã hàng (*)'] = MISA_Excel['Mã hàng']
df_template['Diễn giải'] = MISA_Excel['Diễn giải']
df_template['Mã khách hàng'] = MISA_Excel['Mã KH']
df_template['% thuế GTGT'] = np.where(df_template['Mã hàng (*)'] == 'VCPL',np.nan,df_template['% thuế GTGT'])
df_template['TK thuế GTGT'] = np.where(df_template['Mã hàng (*)'] == 'VCPL',np.nan,df_template['TK thuế GTGT'])

df_template = df_template[~df_template['Diễn giải'].str.contains('Số lượng')]

In [ ]:
df_template.info()

<class 'pandas.core.frame.DataFrame'>
Index: 122 entries, 0 to 161
Data columns (total 57 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Hiển thị trên sổ                                    0 non-null      object 
 1   Hình thức bán hàng                                  0 non-null      object 
 2   Phương thức thanh toán                              0 non-null      object 
 3   Kiêm phiếu xuất kho                                 0 non-null      object 
 4   XK vào khu phi thuế quan và các TH được coi như XK  0 non-null      object 
 5   Lập kèm hóa đơn                                     0 non-null      object 
 6   Đã lập hóa đơn                                      0 non-null      object 
 7   Ngày hạch toán (*)                                  0 non-null      object 
 8   Ngày chứng từ (*)                                   122 non-null    object 
 9   Số c

In [ ]:
# Xuất file input Misa
df_template.to_excel('/content/drive/My Drive/Hung/test/COD_misa.xlsx', index=False)


In [ ]:
Summary = MISA_Excel.copy()

In [ ]:
Summary = Summary.iloc[:, :12]


In [ ]:
# Tạo File Excel tổng hợp từ báo cáo và CTT để check lại số liệu
with pd.ExcelWriter(f'/content/drive/My Drive/Hung/test/Doi Soat COD - {timestamp}.xlsx', engine='xlsxwriter') as writer:
    VNPOST.to_excel(writer, sheet_name='VNPOST', index=False)
    VTPOST.to_excel(writer, sheet_name='VTPOST', index=False)
    dwh.to_excel(writer, sheet_name='BC Cổng thanh toán', index=False)
    dwh_bh.to_excel(writer, sheet_name='BC Tổng hợp bán hàng', index=False)
    DCHECK.to_excel(writer, sheet_name='Check lại', index=False)
    Summary.to_excel(writer, sheet_name='Tổng hợp', index=False)

In [ ]:
# Tính thời gian chạy
elapsed_time = time.time() - start_time
print(f"Thời gian chạy: {elapsed_time:.2f} giây")

Thời gian chạy: 262.25 giây
